In [1]:
import json
with open('data/final_multiturns_wnofun.json','r') as f:
    dataset = json.load(f)

In [40]:
len7 = [a for a in dataset if len(a) == 7]
len8 = [a for a in dataset if len(a) == 8]
len9 = [a for a in dataset if len(a) == 9]
len11 = [a for a in dataset if len(a) == 11]
len13 = [a for a in dataset if len(a) == 13]
len15 = [a for a in dataset if len(a) == 15]

In [134]:
import os
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import requests
import yfinance as yf
import praw
from datetime import datetime,timedelta

from openai import OpenAI
import json




In [236]:
tools = [{"type": "function",
       "function":   {
        "name":'get_earnings',
        "description":'당신은 기업의 실적, 재무재표 또는 현금흐름을 가져오고 분석하는 로봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "symbol":{'type':'string', "description": '실적 데이터를 찾고자 하는 기업의 symbol'},
                'analysis_type':{'type':'string', "description":"""기업의 재무 데이터에서 어떤 정보를 추출하고자 하는지 출력하는 파라미터입니다.
                매출 성장률, 순이익 성장률, 영업이익 성장률 과 같은 기업의 성장성을 요청할경우 'growth'라고 출력,
                얼마나 효율적으로 이익을 창출하는지 수익성과 관련한 질문일경우 'profitability' 출력,
                부채 수준과 재무 건전성과 관련한 질문일경우 'stability' 출력,
                현재 가치와 적정 주가와 관련한 질문일경우 'valuation' 출력,
                현금을 얼마나 잘 창출하는지 확인하여 재무건전성과 관련한 질문일경우 'cashflow' 출력,
                배당 정책과 주식 발행 내역과 관련한 질문일경우 'dividend' 출력,
                비용 절감 능력과 효율성과 관련한 질문일경우 'cost' 출력,
                그외의 재무,실적,현금흐름과 연관이있지만 파라미터를 찾을수없는경우 'NA'라고 출력합니다.

                """},
                "type_":{'type':'string', "description": "연간 데이터 일경우 'yearly' 아닐경우 'quarter' 이라고 표기"},
                "year":{'type':'string', "description": '데이터를 찾고자 하는 연도, 명시 하지 않을경우 가장 최근 연도 로 설정, 1월일경우 , 해당연도에 데이터가 입력되지않을수있음 따라서 최근연도-1 값을 연도라고 표기 '},
                "quarter":{'type':'string', "description": '데이터를 찾고자 하는 분기로 1~4로 이루어진 숫자, 마지막 또는 최근 분기일경우 또는 언급이 없을경우 -1 이라고 표기 '}
            },
            "required":['symbol', 'analysis_type','type_', 'year','quarter'],
            "additionalProperties": False
        },
             "strict": True
    }},
         {"type": "function",
       "function":   {
        "name":'get_consensus',
        "description":'당신은 기업의 (EPS) 컨센서스 데이터 또는, 매수/매도/홀드 의견 목표주가를 가져와서 분석하는 로봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "symbol":{'type':'string', "description": ' 데이터를 찾고자 하는 기업의 symbol'},
                "year":{'type':'string', "description": '데이터를 찾고자 하는 연도, 명시 하지 않을경우 가장 최근 연도 로 설정, 1월일경우 , 해당연도에 데이터가 입력되지않을수있음 따라서 최근연도-1 값을 연도라고 표기 '},
                "quarter":{'type':'string', "description": '데이터를 찾고자 하는 분기로 1~4로 이루어진 숫자, 최근 또는 마지막일경우 -1 이라고 표기 '}
            },
            "required":['symbol', 'year','quarter'],
            "additionalProperties": False
        },
             "strict": True
    }},
        {"type": "function",
         "function":    {
        "name":'get_issues_summarized',
        "description":'당신은 특정 기업 또는 키워드의 이슈검색 챗봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "keyword":{'type':'string', "description": '이슈/현황을 검색하고자 하는 회사명 또는 키워드 in English '},
                "days":{'type':'integer', "description": '검색하고자 하는 기간 예시: 하루, 일주일, 한달, 등등'}

            },
            "required":['keyword','days'],
            "additionalProperties": False
        },
             "strict": True
        }
    },
    {"type": "function",
       "function":   {
        "name":'get_reddit_hotissue',
        "description":'당신은 (금융)시장에서 핫한 이슈를 요약하는 챗봇입니다.',
        'parameters':{
            "type":"object",
            "properties" :{
                "days":{'type':'integer', "description": '검색하고자 하는 기간 예시: 하루, 일주일, 한달,일년 등 **질문에 언급이 없을경우 일주일로 지정'}

            },
            "required":['days'],
            "additionalProperties": False
        },
             "strict": True
    }}]


def get_issues_summarized(keyword ,days):
    redditid = 'mOOCYGbbZZ7_n-x6TucUwQ'
    reddit_pw = '2A3xDNKgeB4ld6wOEUfqOPsObU9WLw'

    # Reddit API 인증
    reddit = praw.Reddit(
        client_id=redditid,
        client_secret=reddit_pw,
        user_agent='test'
    )


    daysago = datetime.utcnow() - timedelta(days=7)
    subreddit = reddit.subreddit('wallstreetbets')

    posts_data = []

    for post in subreddit.search(keyword, sort='hot', limit=500):  # 최근 500개 게시글 확인
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date >= daysago:
            posts_data.append(
                'title :'+ post.title+'\n'+
                'body :' + post.selftext)
    titleandbody = '\n\n'.join(posts_data)

    return titleandbody

def get_reddit_hotissue(days):
    redditid = 'mOOCYGbbZZ7_n-x6TucUwQ'
    reddit_pw = '2A3xDNKgeB4ld6wOEUfqOPsObU9WLw'

    # Reddit API 인증
    reddit = praw.Reddit(
        client_id=redditid,
        client_secret=reddit_pw,
        user_agent='test'
    )


    daysago = datetime.utcnow() - timedelta(days=days)
    subreddit = reddit.subreddit('wallstreetbets')

    posts_data = []
    for post in subreddit.hot(limit=500):
        post_date = datetime.utcfromtimestamp(post.created_utc)
        if post_date >= daysago:
            posts_data.append(
                'title :'+ post.title+'\n'+
                'body :' + post.selftext)
    titleandbody = '\n\n'.join(posts_data)

    return titleandbody


def get_consensus(symbol , year, quarter):
    ticker = yf.Ticker(symbol)
    # 목표주가
    try:
        target_price = ticker.analyst_price_targets
        # buysell = ticker.recommendations
    except:
        target_price='cannot get target price'
        # buysell = 'cannot get recommendations'
    api_key = 'd92779aed523de914055c6b543801a73'
    consensus_url = f'https://financialmodelingprep.com/api/v3/earnings-surprises/{symbol}?apikey={api_key}'
    consensus_response = requests.get(consensus_url).json()

    for consensus in consensus_response:
        consensus['year'] =consensus['date'][:4]
        consensus['EPS_surprise'] = ((consensus['actualEarningResult'] - consensus['estimatedEarning']) /
                                           consensus['estimatedEarning']) * 100


    year_consensus = [a for a in consensus_response if a['year']==year or a['year']==str(int(year)-1)   or a['year']==str(int(year)+1 ) ]
    if year_consensus:
        if quarter =='-1':
            result = [year_consensus[0]]
        else:
            result =  year_consensus
    else:

        result = consensus_response
    if not consensus_response:
        return '컨센서스 데이터를 찾을수 없습니다.'

    result.append({'target_price':target_price})
    return json.dumps(result)
    # return result

def get_earnings(symbol, analysis_type,type_,year, quarter):
    ticker = yf.Ticker(symbol)
    # 주가 데이터 가져오기
    try:
        current_price = ticker.history(period="1d")["Close"].iloc[-1]
    except:
        current_price= 'currently cannot get current price'

    # 재무제표 가져오기
    if type_=='yearly':
        balance_sheet = ticker.balance_sheet
        income_statement = ticker.financials
        cash_flow = ticker.cashflow  # 현금흐름표 가져오기
    else:
        balance_sheet = ticker.quarterly_balance_sheet
        income_statement = ticker.quarterly_financials
        cash_flow = ticker.quarterly_cashflow  # 현금흐름표 가져오기


    earnings = pd.concat([balance_sheet,income_statement])
    earnings = pd.concat([earnings, cash_flow])
    if earnings.empty :
        return "해당 기업의 재무 정보를 찾을수 없습니다."
    total_debt = earnings.T['Total Debt'] if 'Total Debt' in earnings.T.columns else 'None'
    net_debt =  earnings.T['Net Debt'] if 'Net Debt' in earnings.T.columns else 'None'
    total_liabilities = earnings.loc["Total Liabilities Net Minority Interest"].iloc[0]  # 총부채

    # growth 분석
    try:
        total_revenue = earnings.loc["Total Revenue"]
        net_income = earnings.loc["Net Income"]
        revenue_growth_QoQ = ((total_revenue.iloc[0] - total_revenue.iloc[1]) / total_revenue.iloc[1]) * 100  # 매출 성장률 QoQ
        net_income_growth_QoQ = ((net_income.iloc[0] - net_income.iloc[1]) / net_income.iloc[1]) * 100  # 순이익 성장률 QoQ
        normalized_ebitda = float(earnings.loc['Normalized EBITDA'].iloc[0])
        total_assets = float(earnings.loc['Total Assets'].iloc[0])
        invested_capital = float(earnings.loc['Invested Capital'].iloc[0])
        # Variables defined separately
        if analysis_type == 'growth':
            result_dic = {'revenue_growth_QoQ': revenue_growth_QoQ, 'net_income_growth_QoQ': net_income_growth_QoQ,'normalized_ebitda' : normalized_ebitda,'total_assets' : total_assets,'invested_capital':invested_capital}

        # 수익성 (profitability)


        elif analysis_type=='profitability':
            total_revenue = earnings.loc["Total Revenue"]
            gross_profit = earnings.loc['Gross Profit']
            gross_margin = gross_profit/total_revenue
            net_income = earnings.loc["Net Income"]
            net_margin = (net_income.iloc[0] / total_revenue.iloc[0]) * 100  # 순이익률
            stockholders_equity = earnings.loc["Stockholders Equity"].iloc[0]  # 자기자본
            total_assets = earnings.loc['Total Assets']
            roe = (net_income.iloc[0] / stockholders_equity) * 100  # ROE
            roa = (net_income.iloc[0] / total_assets) * 100  # ROA

            EBITDA = earnings.loc['EBITDA']
            EBITDA_margin = EBITDA/total_revenue

            result_dic = {'total_revenue': total_revenue, 'gross_profit': gross_profit,'gross_margin' : gross_margin,'net_income' : net_income,
                          'net_margin':net_margin, 'stockholders_equity':stockholders_equity,'total_assets':total_assets,'roe':roe,'roa':roa,
                         'EBITDA':EBITDA, 'EBITDA_margin':EBITDA_margin}



        elif analysis_type=='stability':
                  ## financial_stability 재무안정성
            debt_to_equity_ratio = total_liabilities / stockholders_equity  # 부채비율

            current_assets = earnings.loc['Current Assets']
            current_liabilities = earnings.loc['Current Liabilities'] # Current Assets / Current Liabilities 유동비율 계산 가능
            inventory = earnings.loc['Inventory'] if  'Inventory' in earnings.T.columns else 0  # (Current Assets - Inventory) / Current Liabilities 당좌 비율
            # (Current Assets - Inventory) / Current Liabilities 당좌 비율
            result_dic = {'debt_to_equity_ratio': debt_to_equity_ratio, 'total_debt': total_debt,'net_debt' : net_debt,
                          'current_assets' : current_assets,
                          'current_liabilities':current_liabilities, 'inventory':inventory}


        # valuation 분석

        # 주식 관련 정보 가져오기

        elif analysis_type=='valuation':
            shares_outstanding = ticker.info["sharesOutstanding"]  # 발행 주식 수
            net_income = income_statement.loc["Net Income"]
            total_assets = earnings.loc["Total Assets"].iloc[0]  # 총자산
            total_liabilities = earnings.loc["Total Liabilities Net Minority Interest"].iloc[0]  # 총부채
            book_value = total_assets - total_liabilities  # 순자산
            eps = net_income.iloc[0] / shares_outstanding  # 주당순이익 (EPS)
            pe_ratio = current_price / eps  # PER
            bvps = book_value / shares_outstanding  # 주당순자산가치
            pb_ratio = current_price / bvps  # PBR

            result_dic = {'shares_outstanding': shares_outstanding,'net_income': net_income.iloc[0],  # 첫 번째 값 사용
                'total_assets': total_assets,'total_liabilities': total_liabilities,'book_value': book_value,'eps': eps,
                'pe_ratio': pe_ratio,'bvps': bvps,'pb_ratio': pb_ratio}
        # cash_flow

        elif analysis_type=='cashflow':


            cash_and_cash_equivalents = earnings.loc["Cash And Cash Equivalents"].iloc[0]
            net_ppe = earnings.loc["Net PPE"].iloc[0]
            result_dic = {'cash_and_cash_equivalents': cash_and_cash_equivalents,'net_ppe': net_ppe}

        # 주식 배당 (Equity & Dividend)

        elif analysis_type=='dividend':
            ordinal_shares_number = earnings.T['Ordinary Shares Number']
            net_income = income_statement.T['Net Income']
            dividends = ticker.dividends
            result_dic = {'ordinal_shares_number': ordinal_shares_number,'net_income':net_income, 'dividends': dividends}


        ## 비용 구조 분석 비용 절감 능력과 효율성을


        elif analysis_type=='cost':
            cost_of_revenue = income_statement.loc['Cost Of Revenue']
            SG_A = income_statement.loc['Selling General And Administration']
            RnD = income_statement.loc['Research And Development']
            result_dic = {'cost_of_revenue': cost_of_revenue,'SG_A':SG_A, 'RnD': RnD}

        ##etc
        # stockholder_equity = earnings.T['Stockholders Equity']
        # longterm_debt = earnings.T['Long Term Debt']
        # total_cap =  earnings.T['Total Capitalization']


        # 예상 성장 값 계산

        elif analysis_type=='expectation':
            current_eps = ticker.info["trailingEps"]  # 현재 EPS
            eps_growth_rate = ticker.info["earningsGrowth"]  # EPS 예상 성장률
            current_net_income = ticker.financials.loc["Net Income"].iloc[0]  # 현재 순이익
            shares_outstanding = ticker.info["sharesOutstanding"]  # 발행 주식 수
            current_revenue = ticker.financials.loc["Total Revenue"].iloc[0]  # 현재 매출
            revenue_growth_rate = ticker.info["revenueGrowth"]  # 매출 성장률
            result_dic = {'current_eps': current_eps,'eps_growth_rate':eps_growth_rate, 'current_net_income': current_net_income,
                         'shares_outstanding':shares_outstanding, 'current_revenue': current_revenue,'revenue_growth_rate':revenue_growth_rate}

        else:
            print('elseelse')
            earningsdf = earnings.T.reset_index()
            earningsdf = earningsdf.rename(columns = {'index':'date'})
            earningsdf['date'] = earningsdf['date'].dt.strftime('%Y-%m-%d')
            result_dic = earningsdf.to_dict('records')
            if quarter=='-1':
              result_dic= result_dic[0]

    except:
        print('exceptexcept')
        earningsdf = earnings.T.reset_index()
        earningsdf = earningsdf.rename(columns = {'index':'date'})
        earningsdf['date'] = earningsdf['date'].dt.strftime('%Y-%m-%d')
        result_dic = earningsdf.to_dict('records')


        if quarter=='-1':
            result_dic= result_dic[0]
    return str(result_dic)

In [220]:
failed =[]
sucess = []
for messages in len7:
    system = messages[0]
    question1 = messages[1]
    toolcall = messages[2]
    toolresponse = messages[3]
    
    
    

In [301]:
def tool_check(messages):
    reddit_hot_query = [{"role": "system", "content": '''당신은 주어진 대화가 올바른 정보를 다루고 있는지 확인하는 로봇입니다. 잘못된 대화일경우 "틀림", 맞는 대화일경우 "맞음" 이라고 출력하세요.\n
    1. 주어진 대화에서 tool_response 가 없을경우, 가상으로 답변을 꾸미지 않았는지 확인하세요. 가상으로 대답하는 대화는 잘못된 대화입니다.\n
    2. 주어진 대화에서 assistant 가 잠시만 기다려 달라고 하는 경우는 잘못된 대화입니다.\n
    3. 주어진 대화에서 tool_response 가 없을경우, 주어진 정보가 없어 대답을 할수없다고 답변 하는 경우는 올바른 대화입니다. \n
    4. 주어진 대화에서 tool_response 가 없을경우, 주어진 정보가 없어 대답을 실패했다고 답변하고 그다음 사용자가 대화가 이어지는 질문을 하는경우는 올바른 대화입니다.
    5. 주어진 대화에서 assistant 가 다른 기업이나 회사에 대한 정보를 제공할수있다고했을대 사용자가 또한번 다른기업이라는 assistant 와 비슷한 맥락의 대화를 하는것은 틀린 대화입니다.
    모든 대화의 자연스러운것이 중요합니다. 자연스럽지않은 경우 틀린 대화 이다.
    예) 
    입력 : {'role': 'assistant',
  'content': 'DBLE의 최근 실적 발표 정보를 찾을 수 없습니다. 기업 심볼을 다시 확인해 주시거나 다른 정보를 제공해 주시면 도움이 되겠습니다.'},
 {'role': 'user', 'content': 'DBLE 외에 다른 기업의 실적 발표 정보를 알고 싶으신가요?'}, 
 출력 : 틀림
    '''}]
    reddit_hot_query.append({"role": "system", "content":str(messages)})
    
    questions_response = client.chat.completions.create(
            model='gpt-4o',
            messages=reddit_hot_query
        )
    return questions_response.choices[0].message.content



In [148]:
multiturn2q = [a for a in len7 if len(a[3]['content']) <= 100 and len(a[4]['content']) > 150]

In [146]:
## 1. 
rightones =[]
# failed =[]
for messages in failedlen7:
    checked = tool_check(messages)
    if '맞음' in checked :
        rightones.append(messages)

In [149]:
failed_toolcall = rightones

In [184]:
tool_check(failed_toolcall[-2])

'맞음'

In [191]:
rightones = [a for a in rightones if '신가요' not in a[-2]['content']]

In [225]:
askedtoolcall= [a for a in rightones if 'tool_call' in a[-1]['content']]

In [242]:
rightonesnotool = [a for a in rightones if '신가요' not in a[-2]['content'] and 'tool_call' not in a[-1]['content']]

In [239]:
import re
import ast
import copy
import pandas as pd
from tqdm import tqdm
multiturntoolcall = []
for toolcallmessages  in tqdm(askedtoolcall):
    messages = copy.deepcopy(toolcallmessages)
    
    matches = re.findall(r'<tool_call>(.*?)</tool_call>', messages[-1]['content'], re.DOTALL)
    toolcallstr  = matches[0].replace('\n','')
    messages[-1]['content'] = '<tool_call>'+toolcallstr +'</tool_call>'
    toolcall = ast.literal_eval(toolcallstr)
    
    name = toolcall['name']
    arguments = toolcall['arguments']
    function_response = call_function(name, arguments)
    if function_response:
    
        tool_response = {'role':'user','content':'<tool_reponse>'+function_response[:2000]+'</tool_reponse>'}
        messages.append(tool_response)
    
        final_response = client.chat.completions.create(
            model='gpt-4o',
            messages=messages,
        )
        messages.append({'role':'assistant', 'content':final_response.choices[0].message.content})
        multiturntoolcall.append(messages)

100%|█████████████████████████████████████████████| 7/7 [00:41<00:00,  5.90s/it]


In [244]:
rightmessages = rightonesnotool+ multiturntoolcall

In [249]:
multiturntoolcall

IndexError: list index out of range

In [250]:
revisedlen7 = multiturn2q + rightmessages

In [258]:
rightmessages[-1] = [{'role': 'system',
  'content': '당신은 다양한 기능을 호출할 수 있는 AI 모델입니다. 사용자의 요청에 따라 특정 함수를 호출하며, 각 함수의 사용 방법과 파라미터는 <tools></tools> XML 태그 안에 명시되어 있습니다. 함수를 호출할 때는 사용자가 제공한 값에 따라 정확히 실행해야 합니다.\n\n사용 가능한 함수와 각 파라미터의 설명은 다음과 같습니다:\n\n1. get_issues_summarized\n   - 설명: 특정 회사나 키워드에 대한 이슈를 검색하고 요약합니다.\n   - 파라미터:\n     - keyword: 검색하고자 하는 회사명 또는 키워드.\n     - days: 검색 기간(일 단위).\n\n2. get_reddit_hotissue\n   - 설명: 금융 시장에서 현재 주목받고 있는 이슈를 요약합니다.\n   - 파라미터:\n     - days: 검색 기간(일 단위).\n\n3. get_earnings\n   - 설명: 기업의 재무재표나 현금흐름을 가져오고 분석합니다. 이전 년도 데이터가 필요한 경우, 해당 데이터를 추가로 호출합니다.\n           함수 호출 시 연도와 분기 설정에 주의해야 합니다. 보통 1분기 실적발표는 4월 중순 ~ 5월 초, 2분기는 7월 중순 ~ 8월초, 3분기는 10월중순~ 11월 초, \n            4분기는 1월 중순 ~2월초에 발표 된다. \n            ## 예시: 1. 오늘 날짜 2025년 2월 18일 , 최근 실적발표의 year  과 quarter 은?\n            year : \'2024\' , quarter: -1 또는, 4 \n   - 파라미터:\n     - symbol: 데이터를 검색하고자 하는 기업 심볼.\n     - analysis_type: 분석 유형(growth, profitability, stability, valuation, cashflow, dividend, cost, NA).\n     - type_: 데이터 타입(yearly 또는 quarter).\n     - year: 데이터를 검색하고자 하는 연도(명시하지 않으면 최근 연도 데이터 참조).\n     - quarter: 데이터를 검색하고자 하는 분기(명시하지 않으면 -1을 입력하여 최근 데이터 조회).\n\n4. get_consensus\n   - 설명: 기업의 EPS 컨센서스 데이터 또는 매수/매도/홀드 의견을 분석합니다. \n     함수 호출 시 연도와 분기 설정에 주의해야 합니다. 컨센서스는 1분기 3월 말 ~ 4월 초, 2분기 6월 말 ~ 7월 초, 3분기는 9월 말 ~ 10월 초, 4분기는 12월 말 ~ 1월초 에 발표된다.\n     ## 예시: 1. 오늘 날짜 2025년 4월 18일 , 최근 컨센서스의 year 과 quarter 은?\n            year : \'2025\' , quarter: 1 또는, -1\n   - 파라미터:\n     - symbol: 데이터를 검색하고자 하는 기업 심볼.\n     - year: 데이터를 검색하고자 하는 연도.\n     - quarter: 데이터를 검색하고자 하는 분기.\n\n함수를 호출할 때는 아래 예시와 같이 <tool_call></tool_call> XML 태그 안에 JSON 객체를 사용하여 함수 이름과 인자들을 명시합니다:\n\n<tool_call>\n{\n    "name": "get_earnings",\n    "arguments": {\n        "symbol": "AAPL",\n        "analysis_type": "growth",\n        "type_": "yearly",\n        "year": "2024",\n        "quarter": "-1"\n    }\n}\n</tool_call>\n\n\n\n**주의사항\n함수를 호출할 때를 제외하고는 모든 응답을 한국어로 제공해야 합니다.\n오늘 날짜는 2022년 09월 20일 입니다.'},
 {'role': 'user', 'content': '이번 최근 PayPal의 매수 의견 수는 몇 개인가요?'},
 {'role': 'assistant',
  'content': '<tool_call>{"name":"get_consensus","arguments": {"symbol": "PYPL", "year": "2022", "quarter": "-1"}}</tool_call>'},
 {'role': 'user',
  'content': '<tool_reponse>[{"date": "2023-11-01", "symbol": "PYPL", "actualEarningResult": 1.3, "estimatedEarning": 1.22, "year": "2023", "EPS_surprise": 6.557377049180333}, {"target_price": {"current": 70.0, "high": 125.0, "low": 70.0, "mean": 95.004, "median": 95.0}}]</tool_response>'},
 {'role': 'assistant',
  'content': 'PayPal의 최근 컨센서스에서는 매수 의견의 수에 대한 자세한 정보가 제공되지 않았습니다. 대신 최근 실적 발표에서 예상 수익이 1.22달러였고 실제 수익은 1.3달러로 수익 서프라이즈가 발생했습니다. 현재 목표 주가는 평균 95.004달러이며, 최고가는 125달러로 나타나 있습니다. 매수 의견 개수에 대한 정보가 더 필요하시면 추가적인 리소스를 참고해 주세요.'},
 {'role': 'user', 'content': 'EPS 서프라이즈는 어떻게 되나요?'},
 {'role': 'assistant',
  'content': '2023년 PayPal(PYPL)의 EPS 서프라이즈는 6.56%입니다. 이는 예상된 수익 대비 실제 수익이 더 높게 나타남에 따라 발생한 서프라이즈입니다.'},
 {'role': 'user', 'content': '목표 주가는 얼마인가요?'},
 {'role': 'assistant',
  'content': 'PayPal(PYPL)의 목표 주가는 평균적으로 95.004달러이며, 최고 목표 주가는 125달러, 최저 목표 주가는 70달러입니다. 현재 목표 주가는 70달러로 설정되어 있습니다.'}]


In [337]:
toolresponselen9 = [a for a in len9 if 'tool_response' in a[-2]['content'] and len(a[-2]['content']) > 100]

In [302]:
notoolresponselen9 = [a for a in len9 if 'tool_response' in a[-2]['content'] and len(a[-2]['content']) < 100]

In [322]:
## 1. 
rightoneslen9 =[]
failedlen9 =[]
for messages in notoolresponselen9:
    checked = tool_check(messages)
    if '맞음' in checked :
        rightoneslen9.append(messages)
    else:
        failedlen9.append(messages)

In [326]:
rightmessages += rightoneslen9

In [341]:
rightmessages += toolresponselen9

In [336]:
## 1. 
rightoneslen9 =[]
failedlen9 =[]
for messages in notoolresponselen11:
    checked = tool_check(messages)
    if '맞음' in checked :
        rightoneslen9.append(messages)
    else:
        failedlen9.append(messages)

219

In [352]:
rightmessages += len11

In [357]:
import pickle
with open('multiturn_revised0418.pkl','wb') as f:
    pickle.dump(rightmessages, f)